In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.datasets import imdb
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Dropout, BatchNormalization, GRU
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Bidirectional,LSTM,Lambda, GRU
from tensorflow.keras.layers import Permute,GlobalMaxPool1D,Concatenate, Dense, BatchNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
%matplotlib inline
from skimage.io import imread
from skimage import color
from skimage.filters import threshold_otsu,gaussian
from skimage.transform import resize
from skimage import measure
import re

In [4]:

    
def scale_csvto_1D(test,lead_no,folder_name,parent):
    target=folder_name[0:2]
    scaler = MinMaxScaler() #scaling
    fit_transform_data = scaler.fit_transform(test)
    Normalized_Scaled=pd.DataFrame(fit_transform_data[:,0], columns = ['X'])
    Normalized_Scaled=Normalized_Scaled.T
    Normalized_Scaled['Target']=target
    #scaled_data to CSV
    if (os.path.isfile('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent))):
      Normalized_Scaled.to_csv('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent), mode='a', header=False,index=False)
    else:
      Normalized_Scaled.to_csv('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent,folder_name=folder_name),index=False)


def extract_signal_leads(Leads,folder_name,parent):
  for x,y in enumerate(Leads):
    fig1 , ax1 = plt.subplots()
    grayscale = color.rgb2gray(y)
    blurred_image = gaussian(grayscale,sigma=0.7)
    global_thresh = threshold_otsu(blurred_image)
    binary_global = blurred_image < global_thresh

    #resize image
    if x!=12:
      binary_global = resize(binary_global, (300, 450))

    ax1.imshow(binary_global,cmap="gray")
    ax1.axis('off')
    ax1.set_title("pre-processed Leads {} image".format(x+1))
    plt.close('all')
    plt.ioff()
    fig1.savefig('{parent}/{folder_name}/Lead_{x}_preprocessed_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent))

    fig7 , ax7 = plt.subplots()

    contours = measure.find_contours(binary_global,0.8)
    contours_shape = sorted([x.shape for x in contours])[::-1][0:1]
    for contour in contours:
      if contour.shape in contours_shape:
        test = resize(contour, (255, 2))
        ax7.plot(test[:, 1], test[:, 0],linewidth=1,color='black')
    ax7.axis('image')
    ax7.set_title("Contour {} image".format(x+1))
    plt.close('all')
    plt.ioff()
    fig7.savefig('{parent}/{folder_name}/Lead_{x}_Contour_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent))
    lead_no=x
    scale_csvto_1D(test,lead_no,folder_name,parent)



def Convert_Image_Lead(image_file,parent_folder):
    #read the image
    image=imread('{parent}/{image_file}'.format(parent=str(parent_folder),image_file=str(image_file)),plugin='matplotlib')
    #dividing the ECG leads from 1-13 
    Lead_1 = image[300:600, 150:643]
    Lead_2 = image[300:600, 646:1135]
    Lead_3 = image[300:600, 1140:1626]
    Lead_4 = image[300:600, 1630:2125]
    Lead_5 = image[600:900, 150:643]
    Lead_6 = image[600:900, 646:1135]
    Lead_7 = image[600:900, 1140:1626]
    Lead_8 = image[600:900, 1630:2125]
    Lead_9 = image[900:1200, 150:643]
    Lead_10 = image[900:1200, 646:1135]
    Lead_11 = image[900:1200, 1140:1626]
    Lead_12 = image[900:1200, 1630:2125]
    Lead_13 = image[1250:1480, 150:2125]
    Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12,Lead_13]

    folder_name= re.sub('.jpg', '',image_file)

    for x,y in enumerate(Leads):
      fig , ax = plt.subplots()
      ax.imshow(y)
      ax.axis('off')
      ax.set_title("Leads {0}".format(x+1))
      if (os.path.exists(parent_folder+'/'+folder_name)):
        pass
      else:
        os.makedirs(parent_folder+'/'+folder_name)
      plt.close('all')
      plt.ioff()
      fig.savefig('{parent}/{folder_name}/Lead_{x}_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent_folder))

    extract_signal_leads(Leads,folder_name,parent_folder)

In [ ]:
normal_parent_dir = '/content/drive/MyDrive/Dataset/Normal'
abnormal_parent_dir = '/content/drive/MyDrive/Dataset/HB'
MI_parent_dir = '/content/drive/MyDrive/Dataset/MI'
MI_history_parent_dir = '/content/drive/MyDrive/Dataset/PMI'

Types_ECG = {'Abnormal_hear_beat':abnormal_parent_dir,'MI':MI_parent_dir,'History_MI':MI_history_parent_dir,'Normal':normal_parent_dir}

for types,folder in Types_ECG.items():
  for files in os.listdir(folder):
    with open(os.path.join(folder, files), 'r') as f:
      Convert_Image_Lead(files,folder)

In [4]:
NORMAL_=[]
MI_=[]
PMI_=[]
HB_=[]

normal = './CSV_files/CSV_files/Normal'
abnormal = './CSV_files/CSV_files/HB'
MI = './CSV_files/CSV_files/MI'
MI_history = './CSV_files/CSV_files/PMI'


Types_ECG = {'normal':normal,'Abnormal_heart_beat':abnormal,'MI':MI,'History_MI':MI_history}

for types,folder in Types_ECG.items():
  for files in os.listdir(folder):
    if types=='normal':
      NORMAL_.append(files)
    elif types=='Abnormal_heart_beat':
      HB_.append(files)
    elif types=='MI':
      MI_.append(files)
    elif types=='History_MI':
      PMI_.append(files)

NORMAL_.sort()
HB_.sort()
MI_.sort()
PMI_.sort()
MI_

['scaled_data_1D_1.csv',
 'scaled_data_1D_10.csv',
 'scaled_data_1D_11.csv',
 'scaled_data_1D_12.csv',
 'scaled_data_1D_13.csv',
 'scaled_data_1D_2.csv',
 'scaled_data_1D_3.csv',
 'scaled_data_1D_4.csv',
 'scaled_data_1D_5.csv',
 'scaled_data_1D_6.csv',
 'scaled_data_1D_7.csv',
 'scaled_data_1D_8.csv',
 'scaled_data_1D_9.csv']

In [5]:
for x in range(len(MI_)):
  df1=pd.read_csv('./CSV_files/CSV_files/Normal/{}'.format(NORMAL_[x]))
  df2=pd.read_csv('./CSV_files/CSV_files/HB/{}'.format(HB_[x]))
  df3=pd.read_csv('./CSV_files/CSV_files/MI/{}'.format(MI_[x]))
  df4=pd.read_csv('./CSV_files/CSV_files/PMI/{}'.format(PMI_[x]))
  final_df = pd.concat([df1,df2,df3,df4],ignore_index=True)
  final_df.to_csv('./CSV_files/CSV_files/Combined_IDLead_{}.csv'.format(x+1))
 

In [6]:

df=pd.read_csv('./CSV_files/CSV_files/Combined.csv')
df['Target'].unique()
df.drop(columns=['Unnamed: 0.1','Unnamed: 0'],inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,Target,lead
0,0.712904,0.715179,0.725315,0.726735,0.722593,0.722546,0.688978,0.638312,0.579908,0.513230,...,0.565029,0.622996,0.660655,0.657961,0.662683,0.654614,0.648513,0.652790,No,1
1,0.611084,0.661575,0.695790,0.741113,0.716666,0.595794,0.425022,0.286457,0.425022,0.611384,...,0.042690,0.165850,0.363445,0.549460,0.539346,0.522272,0.491668,0.454949,No,1
2,0.919093,0.863918,0.792460,0.709667,0.630708,0.596958,0.647904,0.714959,0.788015,0.849807,...,0.633119,0.563701,0.520799,0.546429,0.616661,0.698093,0.781572,0.846886,No,1
3,0.813616,0.798023,0.769452,0.721033,0.664635,0.638127,0.657189,0.648730,0.622964,0.575504,...,0.567919,0.579769,0.573874,0.584347,0.634748,0.696867,0.727551,0.745446,No,1
4,0.728449,0.680755,0.619010,0.645367,0.681570,0.732488,0.758448,0.750660,0.728282,0.707928,...,0.664539,0.667226,0.637064,0.593287,0.545503,0.515049,0.563257,0.633581,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14068,0.746390,0.674968,0.590248,0.514980,0.445196,0.468917,0.556181,0.651794,0.733962,0.778304,...,0.540125,0.445237,0.375814,0.356738,0.408993,0.491161,0.575562,0.654269,PM,13
14069,0.579755,0.590110,0.610995,0.621094,0.589600,0.574719,0.612814,0.648491,0.616001,0.565317,...,0.542677,0.565884,0.527370,0.516367,0.528891,0.542748,0.524488,0.516134,PM,13
14070,0.744649,0.731480,0.707809,0.696605,0.718236,0.741063,0.729558,0.730036,0.687073,0.608270,...,0.632770,0.649485,0.651540,0.625351,0.609527,0.626707,0.653569,0.671543,PM,13
14071,0.382282,0.327976,0.302907,0.330239,0.369734,0.397337,0.409298,0.420891,0.401168,0.411686,...,0.338188,0.328135,0.317905,0.292583,0.238989,0.231455,0.269095,0.318595,PM,13


In [7]:

encode_target_label = df.groupby('Target').ngroup().rename("target").to_frame()
test_final  = df.merge(encode_target_label, left_index=True, right_index=True)
test_final.drop(columns=['Target'],inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,lead,target
0,0.712904,0.715179,0.725315,0.726735,0.722593,0.722546,0.688978,0.638312,0.579908,0.513230,...,0.565029,0.622996,0.660655,0.657961,0.662683,0.654614,0.648513,0.652790,1,2
1,0.611084,0.661575,0.695790,0.741113,0.716666,0.595794,0.425022,0.286457,0.425022,0.611384,...,0.042690,0.165850,0.363445,0.549460,0.539346,0.522272,0.491668,0.454949,1,2
2,0.919093,0.863918,0.792460,0.709667,0.630708,0.596958,0.647904,0.714959,0.788015,0.849807,...,0.633119,0.563701,0.520799,0.546429,0.616661,0.698093,0.781572,0.846886,1,2
3,0.813616,0.798023,0.769452,0.721033,0.664635,0.638127,0.657189,0.648730,0.622964,0.575504,...,0.567919,0.579769,0.573874,0.584347,0.634748,0.696867,0.727551,0.745446,1,2
4,0.728449,0.680755,0.619010,0.645367,0.681570,0.732488,0.758448,0.750660,0.728282,0.707928,...,0.664539,0.667226,0.637064,0.593287,0.545503,0.515049,0.563257,0.633581,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14068,0.746390,0.674968,0.590248,0.514980,0.445196,0.468917,0.556181,0.651794,0.733962,0.778304,...,0.540125,0.445237,0.375814,0.356738,0.408993,0.491161,0.575562,0.654269,13,3
14069,0.579755,0.590110,0.610995,0.621094,0.589600,0.574719,0.612814,0.648491,0.616001,0.565317,...,0.542677,0.565884,0.527370,0.516367,0.528891,0.542748,0.524488,0.516134,13,3
14070,0.744649,0.731480,0.707809,0.696605,0.718236,0.741063,0.729558,0.730036,0.687073,0.608270,...,0.632770,0.649485,0.651540,0.625351,0.609527,0.626707,0.653569,0.671543,13,3
14071,0.382282,0.327976,0.302907,0.330239,0.369734,0.397337,0.409298,0.420891,0.401168,0.411686,...,0.338188,0.328135,0.317905,0.292583,0.238989,0.231455,0.269095,0.318595,13,3


In [8]:
X = test_final.iloc[:,0:-1]
y=test_final.iloc[:,-1]

In [9]:
y.shape

(14073,)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [11]:
X_train = np.array(X_train).reshape((X_train.shape[0],X_train.shape[1],1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1],1))

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)


In [12]:
inputs = Input(shape=(X_train.shape[1], 1))
lstm_layer = LSTM(256, return_sequences=True)(inputs)
lstm_layer = Flatten()(lstm_layer)
outputs = Dense(4, activation='softmax')(lstm_layer)

model_lstm = Model(inputs, outputs)
model_lstm.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256, 256)       │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       262,148 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 526,340 (2.01 MB)

 Trainable params: 526,340 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)
best_model = ModelCheckpoint('./best_lstm_model.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
adam = Adam(learning_rate=0.01)
model_lstm.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
lstm_h = model_lstm.fit(X_train, y_train, batch_size=16, validation_data=(X_test, y_test), epochs=20, callbacks=[es, best_model, lr_schedule])

In [ ]:
acc = lstm_h.history['accuracy']
val_acc = lstm_h.history['val_accuracy']

loss = lstm_h.history['loss']
val_loss = lstm_h.history['val_loss']


plt.figure(figsize=(12, 12))

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Acc.', fontsize=16, weight='bold')


plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')

plt.show()

In [23]:
lstm_best = load_model('./best_lstm_model.keras')
lstm_acc = lstm_best.evaluate(X_test, y_test, verbose=0)[1]
print("Test Acc.: {:.3f}%".format(lstm_acc * 100))

Test Acc.: 87.139%


In [ ]:
y_pred = np.array(list(map(lambda x: np.argmax(x), lstm_best.predict(X_test))))
y_test1 = y_test.idxmax(axis=1)
cm = confusion_matrix(y_test1, y_pred)
print(classification_report(y_test1, y_pred))

In [13]:

inputs = Input(shape=(X_train.shape[1], 1))
gru_layer = GRU(256, return_sequences=True)(inputs)
gru_layer = Flatten()(gru_layer)
outputs = Dense(4, activation='softmax')(gru_layer)
model_gru= Model(inputs, outputs)
model_gru.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 256, 256)       │       198,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │       262,148 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 461,060 (1.76 MB)

 Trainable params: 461,060 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
adam = Adam(learning_rate=0.001)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc = ModelCheckpoint('./best_gru_model.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

model_gru.compile(optimizer=adam,
                  loss=['categorical_crossentropy'],
                  metrics=['accuracy'])

gru_h = model_gru.fit(X_train, y_train,
                   batch_size=16,
                   validation_data=(X_test, y_test),
                   epochs=20,
                   callbacks=[es, mc, lr_schedule])

In [ ]:
acc = gru_h.history['accuracy']
val_acc = gru_h.history['val_accuracy']

loss = gru_h.history['loss']
val_loss =gru_h.history['val_loss']

plt.figure(figsize=(12, 12))

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('GRU - Training & Validation Acc.', fontsize=16, weight='bold')


plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('GRU - Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')

plt.show()

In [ ]:
gru_best = load_model('./best_gru_model.keras')
gru_acc = gru_best.evaluate(X_test, y_test, verbose=0)[1]
print("Test Acc. GRU: {:.3f}%".format(gru_acc * 100))

In [ ]:
y_pred = np.array(list(map(lambda x: np.argmax(x), gru_best.predict(X_test))))
y_test2 = y_test.idxmax(axis=1)
cm = confusion_matrix(y_test2, y_pred)
print(classification_report(y_test2, y_pred))

In [ ]:

model1 = load_model('best_gru_model.keras') 
model2 = load_model('best_lstm_model.keras') 

def predict_proba_keras(model, X):
    y_pred = model.predict(X)
    return y_pred

y_proba_model1 = predict_proba_keras(model1, X_test)
y_proba_model2 = predict_proba_keras(model2, X_test)

y_proba_avg = (y_proba_model1 + y_proba_model2) / 2.0

y_pred = np.argmax(y_proba_avg, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Model Accuracy: {accuracy:.2f}')
